# MongoDB, BM25

# kakao

In [46]:
import json
import pandas as pd
from typing import List,Dict
from tqdm import tqdm
from rank_bm25 import BM25Okapi
import pickle
import requests
from datetime import datetime
import re
import numpy as np

In [47]:
data = json.load(open('./output/seoul/카카오/강남구_output.json'))

In [48]:
def kakao_review(data:Dict):
    output = {}
    id = 0
    review_id = 0
    for key, value in tqdm(data.items()):
        name = value['name'].strip()
        score = value['score'].strip()
        reviews = value['reviews']
        doro_address = key.strip()
        post_processed_reviews = []
        if reviews:
            for review in reviews:
                if review is None:
                    continue
                review = review.split('\n')
                try:
                    date = datetime.strptime(review[5].strip(),'%Y.%m.%d.')#, print(review)
                    date = review[5].strip()
                    text = ' '.join(review[6:-2])
                except:
                    date = datetime.strptime(review[4].strip(),'%Y.%m.%d.')
                    date = review[4].strip()
                    text = ' '.join(review[5:-2])
                    #text = ''
                review_id+=1
                post_processed_reviews.append(dict(date=date, review=text, id=review_id))
        output[key.strip()]=dict(id=id,name=name, score=score, reviews=post_processed_reviews)
        id+=1
    return output

In [49]:
kakao=kakao_review(data)

100%|███████████████████████████████████████████████████████████████████████████| 3304/3304 [00:00<00:00, 36262.15it/s]


## 위 경도 변환

In [50]:
def get_long_lat(address):
    url_front = "http://api.vworld.kr/req/address?"
    url_params = "service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type=road"
    url_address = "&address="
    url_key = "&key="
    auth_key = "7D0E01FD-4222-3E0D-AEB0-9630D650077F"
    url = url_front + url_params + url_address + address + url_key + auth_key
    result = requests.get(url)
    json_data = result.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return [x,y]
    else:
        return None

In [53]:
for i,j in tqdm(kakao.items()):
    long_lat = get_long_lat(i)
    j['geometry']=long_lat

check = []
for i,j in kakao.items():
    if j['geometry'] is None:
        check.append((i,j))

 36%|███████████████████████████▉                                                  | 1184/3304 [01:38<02:56, 12.00it/s]


ConnectionError: HTTPConnectionPool(host='api.vworld.kr', port=80): Max retries exceeded with url: /req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type=road&address=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8%EA%B5%AC%20%ED%85%8C%ED%97%A4%EB%9E%80%EB%A1%9C%20137%20%ED%98%84%EB%8C%80%ED%95%B4%EC%83%81%EB%B9%8C%EB%94%A9%2010%EC%B8%B5&key=7D0E01FD-4222-3E0D-AEB0-9630D650077F (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000028DD2480850>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

## 도로명 주소 > 지번 주소

In [57]:
def get_jibun_addr(data:Dict):
    locations = []
    c=0
    for addr,j in tqdm(data.items()):
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
        headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 2bf374c5b32c92fb51365a34edc4f4c7"}
        try:
            place = requests.get(url, headers = headers).json()['documents']
            jibun = place[0]['address']['address_name']
        except:
            try:
                addr = ' '.join(addr.split()[:-1])
                url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
                place = requests.get(url, headers = headers).json()['documents']
                jibun = place[0]['address']['address_name']
            except:
                jibun = None
                c+=1
        j['jibun_addr']=jibun
    print(f'there is no jibun address - {c}')
    return data

In [58]:
kakao = get_jibun_addr(kakao)

100%|██████████████████████████████████████████████████████████████████████████████| 3129/3129 [03:27<00:00, 15.05it/s]

there is no jibun address - 5


## 정합성 파악
1. null 값 : x
2. jibun 주소가 없는 경우 : 12 
3. geometry가 없는 경우 : 34

In [59]:
check_geo = []
check_jibun = []
for i,j in kakao.items():
    if not j['geometry']:
        check_geo.append(i)
    if not j['jibun_addr']:
        check_jibun.append(i)
    else:
        if j['jibun_addr'] is None:
            check_jibun.append(i)
print(f'no jibun - {len(check_jibun)}')
print(f'no geo - {len(check_geo)}')

no jibun - 5
no geo - 0


In [60]:
# 수기로 검색
for address in check_jibun:
    print(address)
    print(kakao[address])
    check = input()
    kakao[address]['jibun_addr']=check
    print()

서울 강남구 도곡로 516 151-1호, 2호
{'name': '깐부치킨 은마아파트점', 'score': '3.7', 'geometry': ['127.062973708', '37.499128424'], 'reviews': [{'date': '2022.07.02.', 'review': '치킨한마리 배달안해준다는게 말이되나요.. 배달비 준다는데 사이드매뉴를 따로 더 시켜야 배달해주는 양심불량업체', 'id': 2110}, {'date': '2022.03.09.', 'review': '방문했던 깐부치킨 지점 중 가장 맛있었던 곳.', 'id': 2111}, {'date': '2021.10.26.', 'review': '존.맛. 인테리어도 예쁘고, 가까이 있어서 좋아요.ᐟ', 'id': 2112}], 'id': 627, 'jibun_addr': None}


 서울 강남구 대치동 990



서울 강남구 압구정로28길 49 101, 102호
{'name': 'GS25 압구정유림점', 'score': '5.0', 'geometry': ['127.027711329', '37.523255791'], 'reviews': [{'date': '2022.12.12.', 'review': '추천포인트 친절 우산 없어서 급히 들어온 곳인데 사장님 진짜 친절합니다', 'id': 3514}, {'date': '2022.12.10.', 'review': '여기 사장님 너무 친절하시고 즐겁게 말걸어주셔서 이쪽들를일 있으면 꼭 여기로 갑니다 항상 행복하시고 번창하세요!', 'id': 3515}, {'date': '2022.05.24.', 'review': 'GS는 싫지만 여기 점주님은 항상 먼저 즐겁게 인사해주십니다. 늘 감사합니다', 'id': 3516}, {'date': '2021.11.22.', 'review': '사장님이 건네주시는 말 한마디 덕분에 그 날 기분이 좋아지는 곳이예요. 전국 편의점 들어서 제일 친절하신 듯 .. :D', 'id': 3517}, {'date': '2020.06.17.', 'review': '야외 테이블도 있고 주인분도 너무 친절하셔서 좋아요 :)   편의점 리뷰 남겨보기 처음이에요 ㅋㅋㅋ 압구정 오면 나중에 또 방문하려구요!', 'id': 3518}, {'date': '2020.04.16.', 'review': '하나를 사더라도 밝게 인사해주시고 진정성있게 대해주셔서 이 매장은 좋은 에너지가 가득합니다. 가게는 항상 흐트럼없이 정돈이 잘 되어 있고, 기분 나빠서 들어갔다가도 기분 좋아져서 나오게 되는 신비한 곳...^^', 'id': 3519}, {'date': '2020.03.26.', 'review': '주인 아주머니이신 것 같은데, 잠깐 왔다가는 손님들에게도 밝고 활기차게 대해주세요. 인사 끝에 좋은 하루 되시라고 말해 주시는데, 하루가 아름다워질 것 같아요. 감사합니다.', 'id': 3520}, {'date': '2019.11

 서울 강남구 신사동 584-1



서울 강남구 테헤란로27길 16 1, 2층
{'name': '투썸플레이스 역삼역점', 'score': '3.8', 'geometry': ['127.038054855', '37.502438821'], 'reviews': [{'date': '2021.10.01.', 'review': '1층은 좌석이 없는 거나 마찬가지로 생각하면 됨 (1층은 카운터 + 3팀 정도 가능한 좌석) 2층은 코로나 거리두기로 인해 10팀 정도 가능 와이파이 구린건 어느 투썸이나 비슷하니까 괜찮음.   다만 흡연실이 내부에 있는데,  관리가 전혀 이루어지지 않음. 담배냄새가 다 새어나옴.  더구나 마스크 관리는 아예 안함. 흡연실에 대놓고 6명,8명이 낑겨 있어도 테이블에 4인~6인이 모여서 마스크 벗고 있어도 올라와서 살펴보는 직원은 거의 없음. 이따금 올라와도 신경조차 안쓰더라.  위치상 4~6인 팀이 많이 오는 곳이기 때문에 코로나 걸리기 딱 좋은 곳이니 도망가라. 더보기', 'id': 9643}, {'date': '2021.09.30.', 'review': '친절하구 홀케익 사면 아아도 줘욤♡', 'id': 9644}, {'date': '2021.09.25.', 'review': '', 'id': 9645}, {'date': '2021.01.22.', 'review': '', 'id': 9646}], 'id': 2525, 'jibun_addr': None}


 서울 강남구 역삼동 678-10



서울 강남구 강남대로102길 28 1, 2, 지하 1층
{'name': '어퍼앤언더', 'score': '3.1', 'geometry': ['127.027720551', '37.502737230'], 'reviews': [{'date': '2022.12.30.', 'review': '저렇게 맛없는 얼그레이 파운드 처음임 심지어 8천원..? 제가 갔던 카페 중에 가성비도 맛도 하위', 'id': 10598}, {'date': '2022.12.24.', 'review': '', 'id': 10599}, {'date': '2022.12.19.', 'review': '추천포인트 맛 친절 근처에서 찍은 사진 근처에서 찍은 사진 근처에서 찍은 사진 자주 오는데 조금 비싸요', 'id': 10600}, {'date': '2022.12.09.', 'review': '테라스 자리 넘 좋았어요!', 'id': 10601}, {'date': '2022.12.06.', 'review': '추천포인트 맛 분위기 2층 풍경 넘 좋고 실내 분위기 좋고 커피 맛도 좋음! 사람 많아지면 시끄러워서 정신 없어지는 것 빼고 다 좋음 ㅎㅎ', 'id': 10602}, {'date': '2022.11.28.', 'review': '추천포인트 맛 근처에서 찍은 사진 근처에서 찍은 사진 근처에서 찍은 사진 휘낭시에 진짜 잘하시네여 근데 왜케 비싸죠? 휘낭시에 두 개에 6,500원이라닝 ㅠㅠ 비싼데 맛있어서 또 가긴할거예여ㅠㅠ 맛있어요', 'id': 10603}, {'date': '2022.11.18.', 'review': '테라스에서 본 골목풍경이 상당히 예뻤어요. 더 추워지기 전에 기분좋게 마시고 온듯', 'id': 10604}, {'date': '2022.11.07.', 'review': '휘낭시에', 'id': 10605}, {'date': '2022.10.08.', 'review': '추천포인트 분위기 근처에서 찍은 사진 분위기 좋아서 가는 카페입니다… 저거 초코 먹었다가 너무 너무.. 맛이

 서울 강남구 역삼동 618-17



서울 강남구 도곡로77길 8 102, 103호
{'name': '아마당 젤라또 크로플 츄러스', 'score': '5.0', 'geometry': ['127.058647960', '37.498915138'], 'reviews': [{'date': '2022.08.01.', 'review': '기분 좋아져요 . ^^', 'id': 11732}, {'date': '2022.02.05.', 'review': '사장님이 너무 친절하시고 아기자기한 소품이 있어 분위기가 좋아요. 아이스크림도 맛있어요:)', 'id': 11733}, {'date': '2021.12.11.', 'review': '젤라또랑 츄러스도 맛있고 사장님 너무친절하세용 처음왔다고 서비스도 주심ㅠ', 'id': 11734}], 'id': 2965, 'jibun_addr': None}


 서울 강남구 대치동 941


In [61]:
# delete
new_kakao = {}
for i,j in tqdm(kakao.items()):
    if j['jibun_addr']:
        new_kakao[i]=j

100%|█████████████████████████████████████████████████████████████████████████| 3129/3129 [00:00<00:00, 1557187.61it/s]


In [62]:
check_geo = []
check_jibun = []
for i,j in new_kakao.items():
    if not j['geometry']:
        check_geo.append(i)
    if not j['jibun_addr']:
        check_jibun.append(i)
print(f'no jibun - {len(check_jibun)}')
print(f'no geo - {len(check_geo)}')

no jibun - 0
no geo - 0


In [24]:
# geometry가 없는 경우 반대로 검색
# 즉, 지번 주소로 geometry 검색
for i in check_geo:
    print(i)
    print(new_kakao[i])
    check = input()
    geo = get_long_lat(check)
    new_kakao[i]['geometry']=geo

서울 강남구 일원동 717-1 일원역 구내 348-10호
{'id': 319, 'name': '더박스 일원역점', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 일원동 717-1'}


서울 강남구 테헤란로 322 한신인터밸리 24 상가 1층 103호
{'id': 374, 'name': '세븐일레븐 한신점', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 역삼동 707-34'}


 서울 강남구 테헤란로 322 한신인터밸리


서울 강남구 삼성로 150 미도종합상가(남부순환로 3032, 135)
{'id': 581, 'name': '대곡공인중개사사무소', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 대치동 511'}


서울 강남구 대치동 1011-8
{'id': 598, 'name': '은마아파트입구 사거리(1) 대여소', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 대치동 1011-8'}


...

서울 강남구 개포동 169-3
{'id': 2230, 'name': '개포까치공원', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 개포동 169-3'}


서울 강남구 도곡동 455
{'id': 2231, 'name': '도곡까치공원', 'score': '5.0', 'reviews': [{'date': '2022.08.28.', 'review': '근처에서 찍은 사진 근처에서 찍은 사진 그네는 없지만 미끄럼틀 있고 모래놀이도 가능해요', 'id': 7703}], 'geometry': None, 'jibun_addr': '서울 강남구 도곡동 455'}


서울 강남구 도곡동 455 운동기구옆(좌)
{'id': 2232, 'name': '도곡까치어린이공원 음수대2', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 도곡동 455'}


서울 강남구 도곡동 455 운동기구옆(우)
{'id': 2233, 'name': '도곡까치어린이공원 음수대1', 'score': '0.0', 'reviews': [], 'geometry': None, 'jibun_addr': '서울 강남구 도곡동 455'}


KeyboardInterrupt: Interrupted by user

In [67]:
# delete
final_kakao = {}
for i,j in tqdm(new_kakao.items()):
    if j['geometry']:
        final_kakao[i]=j

100%|█████████████████████████████████████████████████████████████████████████| 3129/3129 [00:00<00:00, 1042744.10it/s]


In [80]:
def check_format(data:Dict):
    #ids = []
    #review_ids = []
    new_data = {}
    for i,b in tqdm(data.items()):
        assert type(i)==str
        assert type(b['name'])==str
        assert type(b['score'])==str
        if '.' in b['score']:
            score = b['score']
        else:
            continue
        name = b['name']
        geometry = b['geometry']
        if b.get('jibun_addr'):
            jibun_addr = b['jibun_addr']
        else:
            continue
        new_data[i.strip()]=dict(name=name, score=score, geometry=geometry, jibun_addr=jibun_addr) 
        new_data[i.strip()]['reviews']=[]
        for rev in b['reviews']:
            assert type(rev['review'])==str
            review = rev['review']
            assert type(rev['date'])==str
            try:
                if rev['date'][4]=='.' and rev['date'][7]=='.':
                    date = rev['date']
                    new_data[i.strip()]['reviews'].append(dict(date=date, review=review))
                else:
                    continue
            except:
                continue
            #assert type(rev['id'])==int
            #review_ids.append(b['id'])
    print(f'{len(data)} ---> {len(new_data)}')
    return new_data

In [68]:
final_kakao = check_format(final_kakao)

100%|██████████████████████████████████████████████████████████████████████████| 3129/3129 [00:00<00:00, 164566.04it/s]

3129 ---> 3129


In [69]:
def make_ids(data):
    id = 0
    review_id = 0
    for i,j in data.items():
        j['id']=id
        id+=1
        for rev in j['reviews']:
            rev['id']=review_id
            review_id+=1
    return data

In [70]:
final_kakao = make_ids(final_kakao)

In [71]:
json.dump(final_kakao, open('final_kakao.json','w',encoding='utf-8'), ensure_ascii=False)

## BM25
### BM25 학습 - doro명 주소 

In [75]:
tokenized_corpus = [(i+' '+j['name']).split(' ') for i,j in final_kakao.items()]
tokenized_corpus += [(j['jibun_addr']+' '+j['name']).split(' ') for i,j in final_kakao.items()]
bm25_id_to_db_id = {}
c=0
for i,j in final_kakao.items():
    bm25_id_to_db_id[c]=j['id']
    c+=1
for i,j in final_kakao.items():
    bm25_id_to_db_id[c]=j['id']
    c+=1
bm25 = BM25Okapi(tokenized_corpus)

In [76]:
pickle.dump(bm25, open('kakao_bm25','wb'))
pickle.dump(bm25_id_to_db_id, open('kakao_bm25_id_to_db_id','wb'))

In [59]:
def retrieve(bm25, bm25_id_to_db_id, query):
    tokenized_query = query.split(' ')
    doc_scores = bm25.get_scores(tokenized_query)
    sorted_idx = np.argsort(-doc_scores)
    result = [i for i in sorted_idx]
    output = []
    for i in result:
        tmp = bm25_id_to_db_id[i]
        if tmp not in output:
            output.append(tmp)
    return output

# mango

In [63]:
mango_data = json.load(open('./output/seoul/mango/강남구_output_mango_score.json'))

In [64]:
def mango_review(data:Dict):
    output = {}
    id = 0
    review_id = 0
    for i,a in tqdm(data.items()):
        for j in a:
            if j is None:
                break
            name = j['score'].split('\n')[0]
            score = name.split()[-1]
            try:
                score = str(float(score))
                name = ' '.join(name.split()[:-1])
            except:
                score = '0.0'
            name = re.sub('[()]','', name).strip()
            tmp = j['detailed_info'].split('\n')
            doro_addr = None
            jibun_addr = None
            for tmp_i in tmp:
                if tmp_i.startswith('주소 '):
                    doro_addr = re.sub('주소 ', '', tmp_i).strip()
                elif tmp_i.startswith('지번 '):
                    jibun_addr = re.sub('지번 ','',tmp_i).strip()
            post_processed_reviews = []
            if j['review']:
                for review in j['review']:
                    tmp = review.split('\n')
                    date = tmp[-2]
                    date = re.sub('-','.',date)
                    text = ' '.join(tmp[3:-2])
                    post_processed_reviews.append(dict(id=review_id, review=text, date=date))
                    review_id+=1
            if doro_addr is not None:
                output[doro_addr]=dict(name=name, score = score, doro_addr=doro_addr, jibun_addr=jibun_addr, reviews = post_processed_reviews)
    return output

In [113]:
mango =  mango_review(mango_data)

100%|███████████████████████████████████████████████████████████████████████████| 1325/1325 [00:00<00:00, 15458.93it/s]


In [114]:
for i,j in tqdm(mango.items()):
    long_lat = get_long_lat(i)
    j['geometry']=long_lat

check = []
for i,j in mango.items():
    if j['geometry'] is None:
        check.append((i,j))

100%|████████████████████████████████████████████████████████████████████████████████| 408/408 [00:29<00:00, 13.99it/s]


In [115]:
mango = get_jibun_addr(mango)

100%|████████████████████████████████████████████████████████████████████████████████| 408/408 [00:32<00:00, 12.44it/s]

there is no jibun address - 67


In [125]:
check_jibun=[]
check_geo=[]
for i,j in mango.items():
    if not j['jibun_addr'] :
        check_jibun.append(i)
    else:
        if j['jibun_addr'] is None:
            check_jibun.append(i)
    if not j['geometry'] :
        check_geo.append(i)
print(f'no jibun - {len(check_jibun)}')
print(f'no geo - {len(check_geo)}')

no jibun - 42
no geo - 4


In [126]:
# 수기로 검색
for address in check_jibun:
    print(address)
    print(mango[address])
    check = input()
    mango[address]['jibun_addr']=check
    print()

서울특별시 강남구 논현로87길 41 104-2호
{'name': '죠샌드위치 역삼포스틸점', 'score': '0.0', 'doro_addr': '서울특별시 강남구 논현로87길 41 104-2호', 'jibun_addr': '', 'reviews': [{'id': 6370, 'review': '샌드위치는 실한편! 클럽샌드위치 추천받았어여 5300원이에요. 딱히 맛있지는 않아요. 그냥 엄마가 해준 정도의 맛.. 역삼에 서브웨이가 생겼으면 좋겠어요 차라리 2015-12-07', 'date': '+4'}], 'geometry': ['127.033517148', '37.498832393']}


 서울특별시 강남구 역삼동 735-11 1F



서울특별시 강남구 역삼로 136 1F
{'name': '죠샌드위치', 'score': '0.0', 'doro_addr': '서울특별시 강남구 역삼로 136 1F', 'jibun_addr': '', 'reviews': [], 'geometry': ['127.034510108', '37.494203641']}


 서울특별시 강남구 역삼동 834-44 1F



서울특별시 강남구 압구정로 162 1F
{'name': '에그드랍 압구정점', 'score': '3.3', 'doro_addr': '서울특별시 강남구 압구정로 162 1F', 'jibun_addr': '', 'reviews': [{'id': 24091, 'review': '에그드랍 별 생각없었는데 오랜만에 먹으니까 또 맛있네여? 이삭토스트랑은 또 다른 맛ㅇㅇ 왜 집에서 해먹으면 요런 맛이 안날까 빵 구울 때 버터도 잔뜩 발라주시고 만드시는거 직접 구경하다가 먹으니까 더 맛있는거 같았음 토스트 안에 재료가 다양하게 들어가서 그런지 하나 만드는 데 정성이 꽤 들어가더라구요. 맛있게 잘 먹었습니당ㅇㅇ', 'date': '2022.01.20'}, {'id': 24092, 'review': '달달한 계란샌드위치집 비슷한 가게(에그***)에 비해 달달합니다 아침이나 간식으로 먹기 좋습니다', 'date': '2020.10.04'}, {'id': 24093, 'review': '요기요로 맛집배달을 시켜보았다... 따뜻하고 부드럽고 고소하고 혼자 다했다 완전 맛있음 냠냠 또 시켜먹을꼬야 ', 'date': '2020.03.11'}, {'id': 24094, 'review': '빵이 촉촉하다. 먹기 불편하고 아래로 갈수록 빵밖에 안남아서 아쉽긴하다. 커피를 많이 먹으러 가는곳', 'date': '2019.12.31'}, {'id': 24095, 'review': '먹은 거: 베이컨더블치즈(3,900원) 맛 보통 가성비 좋음. 친절함 보통. 매장 깔끔함.  양은 좀 적은 편. 푹신한 스크램블 에그의 식감이 좋다. 아메리카노를 포함해도 4,800원이라는 훌륭한 가격.', 'date': '2019.10.12'}, {'id': 24096, 'review': '끼니 떼우기 좋은 토스트집! 생각보다 식사시간에 손님이 많아서 조금 기다리실 수 있습니다!! 베이컨이나 햄 무난하게 많이 먹었던 것 같은데 단짠단짠하니 맛있습니다!! 드실 때 소스나 계란 흐르지 않게 조심해서 드셔야할 듯해욯ㅎ

...


서울특별시 강남구 선릉로146길 5 1F
{'name': '현대정육식당', 'score': '3.5', 'doro_addr': '서울특별시 강남구 선릉로146길 5 1F', 'jibun_addr': None, 'reviews': [{'id': 6170, 'review': '새벽에.. 취해서.. 끌려온... 이곳... 어딘지도 모르게 맛있게 먹었다.... 사진 안 찍었으면 어쩔뻔', 'date': '2022.12.01'}, {'id': 6171, 'review': '김치찌개 맛집으로 알려진 곳인데 찌개는 평범하고 그보단 계란말이, 제육볶음이 더 맛있다. 밑반찬이 꽤 정갈하게 나오는 편이라 집밥 같은 백반이 먹고싶다면 추천하고 싶다. 지금은 코로나 덕분에 무뎌졌지만 24시간 영업이 큰 장점. 맨날 점심만 먹다가 저녁시간에 방문해 삼겹살, 찌개를 시도해봤다. 결론은 고기는 고기집에서 먹는걸로! ^^; 2021-12-20', 'date': '+1'}, {'id': 6172, 'review': '제육 김치찌개 계란말이 맛있다 하지만 맛집이라기보다는 걍 좀 잘나가는 동네 식당', 'date': '2019.01.08'}, {'id': 6173, 'review': '갈때마다 항상 똑같은 조합 = 차돌+김치찌개! 24시간 영업이었나 가물가물 기억은 안 나는데 대개 여기를 찾을때는 저녁을 건너뛰어서 밤에 답없이 배고픈 경우라 항상 아주 늦은 시간이었고... 김치찌개의 묵은지가 리얼루다가 밥도둑이당 흐흥 그리고 발렛됩니당', 'date': '2018.08.18'}, {'id': 6174, 'review': '한식이 생각날때 들리는 곳. 여기 김치찌개 정말 좋아해요', 'date': '2018.06.23'}, {'id': 6175, 'review': '김치찌개에 계란말이 굿굿', 'date': '2018.04.13'}, {'id': 6176, 'review': '김치찌개는 제 입맛에는 안맞았는데 끝까지 먹게되는 맛이었어요 목살김치볶음은 단맛이 강하고 매콤했구요 전체적으로 먹을만 


서울특별시 강남구 테헤란로78길 16 1F
{'name': '롤링핀 포스코사거리점', 'score': '0.0', 'doro_addr': '서울특별시 강남구 테헤란로78길 16 1F', 'jibun_addr': None, 'reviews': [{'id': 6187, 'review': '빵순이를 위한 롤링핀 집앞이라 자주 가는데 빵굽는 냄새가 너무 좋아여 오늘은 고구마식빵 픽해왔어요. 빵 종류가 은근많아서 매번 고르기힘들만 실패는 없었네요. 담백한 빵 가득하니 롤링핀식빵종류 드셔보세요. 추천합니다.', 'date': '2022.04.19'}, {'id': 6188, 'review': '시나몬롤. 버터프레첼.  버터프레첼은 짭쪼름하고 버터와 어우러지는 맛이 정말 맛남. 약간 앙버터느낌도 나지만 짠맛이 나서 좀 다른. 빵은 좀 딱딱하고 한입에 다 넣기도 애매하기도해서 조금 먹기 힘들었지만 그래도 맛났음. 시나몬롤은 시나몬향이 엄청 남. 그리고 달달해서 초등학생인 조카도 넘 잘 먹음ㅋㅋ 버터프레첼은 한입먹고 안먹더니 이건 계속 먹었음. 시나몬 향이 강해서 싫어할까 걱정했는데 엄청 좋아함.', 'date': '2019.05.18'}], 'geometry': ['127.055338352', '37.505062614']}



서울특별시 강남구 논현로4길 36 1F
{'name': '고다방', 'score': '0.0', 'doro_addr': '서울특별시 강남구 논현로4길 36 1F', 'jibun_addr': None, 'reviews': [{'id': 6269, 'review': '이름부터 인테리어까지 레트로 감성 가득. 컵도 큰데 가격도 괜찮다. 정말 좋은 카페인데 위치가 아쉬울 따름...', 'date': '2020.02.10'}], 'geometry': ['127.052931756', '37.473668092']}



서울특별시 강남구 압구정로12길 6
{'name': '커피빈 뉴가로수길점', 'score': '0.0', 'doro_addr': '서울특별시 강남구 압구정로12길 6', 'jibun_addr': None, 'reviews': [], 'geometry': None}


KeyboardInterrupt: Interrupted by user

In [128]:
new_mango = {}
# 수기로 검색
for address,j in mango.items():
    if j['jibun_addr'] is not None:
        if j['jibun_addr']:
            new_mango[address]=j

In [130]:
check_jibun=[]
check_geo=[]
for i,j in new_mango.items():
    if not j['jibun_addr'] :
        check_jibun.append(i)
    else:
        if j['jibun_addr'] is None:
            check_jibun.append(i)
    if not j['geometry'] :
        check_geo.append(i)
print(f'no jibun - {len(check_jibun)}')
print(f'no geo - {len(check_geo)}')

no jibun - 0
no geo - 2


In [134]:
# 수기로 검색
for address in check_geo:
    print(address)
    print(new_mango[address])
    check1 = input()
    check2 = get_long_lat(check1)
    print(check2)
    mango['geometry'] = check2     

서울특별시 강남구 압구정로46길 46 현대카드 쿠킹라이브러리 1F
{'name': '현대카드 쿠킹 라이브러리델리', 'score': '4.3', 'doro_addr': '서울특별시 강남구 압구정로46길 46 현대카드 쿠킹라이브러리 1F', 'jibun_addr': '서울 강남구 신사동 645-22', 'reviews': [{'id': 26209, 'review': '괜찮다. 간장달걀 +아보카도+명란. 아보카도와 명란은 각각 추가금 발생. 근데 이걸 먹으러 올 내용은 아닐듯', 'date': '2022.08.31'}, {'id': 26210, 'review': '<테라스 카페 맛집> 우리는 점심을 먹은터라 1층 카페 입성 현대카드로 결제시 10프로 할인 커피신선하고 산미적당 밸런스 굿 무엇보다 테라스가 좋았고 다양한 쿠킹북을 읽으며 좋은시간을 보냈다', 'date': '2022.06.02'}, {'id': 26211, 'review': "의외의 까눌레 맛집이라 쓰는 리뷰. 얼그레이 까눌레(3,000), 바스크 치즈케이크 (7,000), 카페라떼(6,000) 주문.  여기서 저녁도 먹고 바로 디저트도 먹었다. 현대카드로 결제 시 전 메뉴 10%할인되고, 1층 델리는 현대카드 비소지자도 이용 가능하다. 얼그레이를 그렇게 좋아하지 않는다는 걸 얼마 전에 알아서 까눌레는 조금 고민했는데 잘 시킨 것 같다.  자리로 가져다주는데 나이프 없이 포크만 두 개 준 건 조금 아쉬웠다. (많이 안 먹어봤지만) 지금까지 내 까눌레 원탑은 제주도 토투가였는데, 여기 까눌레를 먹으며 그곳이 생각났다. 겉면이 일정하게 얇고 과자처럼 바삭하며, 안은 촉촉했다. 좀만 두꺼우면 자르기 힘들고 바삭이 아니라 딱딱해지는데 그 선을 잘 지켰다. 또 얼그레이 향이 그렇게 강하지는 않아 거부감 없이 먹었다. 라떼는 폴바셋 원두를 쓴다. 온라인 메뉴판에는 싱글오리진도 있던데 그걸로 시켜본다는 걸 깜빡했다. 바스크 치즈케이크는 겉면은 잘 탔는데 빵 부분이 바스크치고 쫀쫀한 식감은 아니라 조금 아쉬웠다. 

 서울특별시 강남구 압구정로46길 46


['127.035660120', '37.525645966']
서울특별시 강남구 테헤란로 322 한신인터밸리 3F
{'name': '커피코트 선릉 한신인터밸리점', 'score': '0.0', 'doro_addr': '서울특별시 강남구 테헤란로 322 한신인터밸리 3F', 'jibun_addr': '서울 강남구 역삼동 707-34', 'reviews': [{'id': 7842, 'review': '지난 두 번의 방문에서 로스터즈 같은 가게만의 특별한 개성이 묻어나는 맛과 향의 커피는 아니더라도 특정 품종의 원두에서 기대하는 맛과 향을 충실히 내면서 가격도 좋고 미국 느낌으로 에어 포트에서 따라서 마시는 게 추억 뿜뿜이었던 게 좋았던지라 포인트 적립도 등록을 했었음.  가게 위치도 그렇고 이 가게만의 차별점인 에어 포트를 이용해 주변 직장인들을 상대로 대용량 테이크 아웃 서비스도 제공하길래 집에서 마시는 네스프레소는 좀 오버해서 거의 백여가지 맛을 돌아가면서 마시고 있지만 에디오피아 말고는 식상한 느낌이던 중 에어 포트 테이크 아웃 서비스가 궁금해서 이용해보기로 함.  토요일은 평일보단 짧게 영업하지만 그래도 혹시나 해서 영업중인 걸 확인도 하고 에어 포트 테이크 아웃 서비스 문의도 할 겸 전활해보니 늘 친절하신 젊은 사장님 내진 점장님이 받으셨고 원랜 토요일에도 영업을 했었는데 주말에 지방에서 올라오신 손님들이 마스크도 착용을 하지 않고 이용하다 보니 문제가 있었어서 이젠 토요일엔 영업을 하지 않는데 오늘은 단체 손님 주문 관계로 나와 계시다고 하심.  에어 포트 테이크 아웃 서비스 이용 방법을 문의를 하니 핫 드립 커피는 7천 원에, 아이스나 콜드 브루는 13천 원에 커피 2.2리터가 든 에어 포트를 준비해 드리고 가져가셨다가 2-3일 내 또는 사정에 따라 조금 더 지나서 가게에 반납해 주시면 된다고 하심.  핫 드립 커피는 준비하는데 시간이 조금 더 걸리는지라 최소 픽업 10분 전에 아이스나 콜드 브루는 더 빨리 준비가 가능하다고 하시고 12시까지 가게에 계

 서울특별시 강남구 테헤란로 322


['127.046582408', '37.503235582']


In [69]:
final_mango = {}
id = 0
review_id = 0
for i,j in new_mango.items():
    i = re.sub('서울특별시', '서울', i)
    i = re.sub('강남구', '강남', i)
    j['jibun_addr'] = re.sub('서울특별시', '서울', j['jibun_addr'])
    j['jibun_addr'] = re.sub('강남구', '강남', j['jibun_addr'])
    j['id']=id
    id+=1
    for rev in j['reviews']:
        rev['id']=review_id
        review_id+=1
    final_mango[i] = j

In [84]:
# save
json.dump(final_mango, open('final_mango.json','w',encoding='utf-8'), ensure_ascii=False)

In [85]:
tokenized_corpus = [(i+' '+j['name']).split(' ') for i,j in final_mango.items()]
tokenized_corpus += [(j['jibun_addr']+' '+j['name']).split(' ') for i,j in final_mango.items()]
bm25_id_to_db_id = {}
c=0
for i,j in final_mango.items():
    bm25_id_to_db_id[c]=j['id']
    c+=1
for i,j in final_mango.items():
    bm25_id_to_db_id[c]=j['id']
    c+=1
bm25 = BM25Okapi(tokenized_corpus)

In [86]:
pickle.dump(bm25, open('mango_bm25','wb'))
pickle.dump(bm25_id_to_db_id, open('mango_bm25_id_to_db_id','wb'))

In [81]:
final_mango=json.load(open('final_mango.json','r',encoding='utf-8'))

In [83]:
final_mango = check_format(final_mango)
final_mango = make_ids(final_mango)

100%|████████████████████████████████████████████████████████████████████████████| 387/387 [00:00<00:00, 193629.45it/s]

387 ---> 387
